# 2022 Running Analysis


In [6]:
import pandas as pd

from stravadataanalysis.src import file_utils

json_activities_2022 = file_utils.read_json_files("../data/2022/activities")
run_activities_2022 = pd.DataFrame.from_dict(json_activities_2022)
run_activities_2022 = run_activities_2022[run_activities_2022['type'] == 'Run']


## 2022 Highlight

In [8]:
from datetime import datetime

from stravadataanalysis.src import data_display
from stravadataanalysis.src.strava import highlights

run_activities_2022['start_time'] = run_activities_2022['start_date_local'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())

highlights_requests = [
    highlights.Highlight('max_distance', 'distance', display_function=data_display.m_as_km),
    highlights.Highlight('max_duration', 'elapsed_time', display_function=lambda x: data_display.seconds_as_hhmmss(int(x))),
    highlights.Highlight('max_elevation_gain', 'total_elevation_gain', display_function=data_display.m),
    highlights.Highlight('earlier_run', 'start_time', ascending_sort_order=True),
    highlights.Highlight('max_average_speed', 'average_speed', data_display.mps_as_minpkm),
    highlights.Highlight('max_top_heartrate', 'max_heartrate', data_display.bpm),
]

columns = ["highlight", "value", "name", "start_date_local", "id"]

highlights_2022 = []
for highlight_request in highlights_requests:
    highlight = highlight_request.transform(run_activities_2022)
    highlights_2022.append(highlight)

pd.DataFrame(highlights_2022, columns=columns)

,highlight,value,name,start_date_local,id
15,max_distance,21.34km,Course à pied l'après-midi,2022-03-13T16:21:44Z,6819196468
15,max_duration,01:50:38,Course à pied l'après-midi,2022-03-13T16:21:44Z,6819196468
21,max_elevation_gain,263m,Henvic / Sentier Côtier à Carantec,2022-04-17T17:00:17Z,6997761739
37,earlier_run,06:51:23,Course à pied le matin,2022-07-12T06:51:23Z,7454700738
26,max_average_speed,4:45min/km,Course à pied matinale,2022-05-07T10:58:42Z,7101290577
0,max_top_heartrate,192bpm,Course à pied sous la pluie,2022-01-03T12:01:08Z,6465654440


## Segments
Top 10 segments of the year :

In [9]:
import itertools

from stravadataanalysis.src.strava import segments

activities_segments = run_activities_2022['segment_efforts'].apply(segments.extract_segments)
segments_2022 = pd.DataFrame(itertools.chain.from_iterable(activities_segments.values))

top_10_segments = segments_2022.groupby(['id']).agg(count=('id', 'count'), name=('name', 'min'))[['count', 'name']].reset_index().sort_values(
    by=('count'), ascending=False).head(10)
display(top_10_segments)



,id,count,name
84,20310261,30,passage à côté du stade jusqu'au terrain de te...
42,13276220,17,Retour depuis pont
5,5403095,15,1km Chezine
81,20263817,15,vers aire de jeu du parc de la Chézine
79,20263702,14,Chézine partie sud du circuit principal
15,6899362,13,Montée de l'Avenue des Acacias
94,22650322,13,Bassin de la Chezine --> 2ème pont
74,19642112,13,chez moi
111,27607808,12,Pont Jules César --> Bd du Massacre
83,20310176,12,La Gourmette vers bout du lac
